In [29]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import cv2

In [30]:
df = pd.read_csv('labels.csv')

In [31]:
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\Cars0.xml,226,419,125,173
1,./images\Cars1.xml,134,262,128,160
2,./images\Cars10.xml,140,303,5,148
3,./images\Cars100.xml,175,214,114,131
4,./images\Cars101.xml,167,240,202,220


In [32]:
import xml.etree.ElementTree as xet

In [33]:
filename = df['filepath'][0]
filename

'./images\\Cars0.xml'

In [34]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [35]:
getFilename(filename)

'./images\\Cars0.png'

In [36]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images\\Cars0.png',
 './images\\Cars1.png',
 './images\\Cars10.png',
 './images\\Cars100.png',
 './images\\Cars101.png',
 './images\\Cars102.png',
 './images\\Cars103.png',
 './images\\Cars104.png',
 './images\\Cars105.png',
 './images\\Cars106.png',
 './images\\Cars107.png',
 './images\\Cars108.png',
 './images\\Cars109.png',
 './images\\Cars11.png',
 './images\\Cars110.png',
 './images\\Cars111.png',
 './images\\Cars112.png',
 './images\\Cars113.png',
 './images\\Cars114.png',
 './images\\Cars115.png',
 './images\\Cars116.png',
 './images\\Cars117.png',
 './images\\Cars118.png',
 './images\\Cars119.png',
 './images\\Cars12.png',
 './images\\Cars120.png',
 './images\\Cars121.png',
 './images\\Cars122.png',
 './images\\Cars123.png',
 './images\\Cars124.png',
 './images\\Cars125.png',
 './images\\Cars126.png',
 './images\\Cars127.png',
 './images\\Cars128.png',
 './images\\Cars129.png',
 './images\\Cars13.png',
 './images\\Cars130.png',
 './images\\Cars131.png',
 './images\\Cars132.

# Verify image and Output


In [37]:
file_path = image_path[3]
file_path

'./images\\Cars100.png'

In [38]:
img = cv2.imread(file_path)
cv2.rectangle(img,( 175,114),(214,131),(0,255,255),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Data Preprocessing

In [39]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [40]:
labels = df.iloc[:,1:].values
labels

array([[226, 419, 125, 173],
       [134, 262, 128, 160],
       [140, 303,   5, 148],
       ...,
       [ 98, 297, 102, 191],
       [ 85, 247, 196, 259],
       [ 70, 262, 198, 271]], dtype=int64)

In [41]:
labels.shape

(433, 4)

In [42]:
data = []
output = []

for ind in range(len(image_path)):
    
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape

    # preprocessing

    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0
    # normalization to labels

    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax)
    
    data.append(norm_load_image_arr)
    output.append(label_norm)

# Spliting the Data in Train and Testing

In [43]:
X = np.array(data,dtype = np.float32)
Y = np.array(output,dtype = np.float32)

In [44]:
X.shape

(433, 224, 224, 3)

In [45]:
Y.shape

(433, 4)

In [46]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

# Deep Learning Model

In [47]:
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2, InceptionV3
from tensorflow.keras.layers import Dense,Dropout,Flatten,Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [48]:
inception_resnet = InceptionResNetV2(weights='imagenet',include_top = False ,
                                     input_tensor = Input(shape=(224, 224, 3)))
inception_resnet.trainable = False

In [49]:
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation='relu')(headmodel)
headmodel = Dense(250,activation='relu')(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

In [50]:
model = Model(inputs = inception_resnet.input,outputs=headmodel)

In [51]:
# compiling the model

model.compile(loss = 'mse',optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

 batch_normalization_4 (Bat  (None, 52, 52, 192)          576       ['conv2d_4[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_4 (Activation)   (None, 52, 52, 192)          0         ['batch_normalization_4[0][0]'
                                                                    ]                             
                                                                                                  
 max_pooling2d_1 (MaxPoolin  (None, 25, 25, 192)          0         ['activation_4[0][0]']        
 g2D)                                                                                             
                                                                                                  
 conv2d_8 (Conv2D)           (None, 25, 25, 64)           12288     ['max_pooling2d_1[0][0]']     
          

# Model Training

In [52]:
from tensorflow.keras.callbacks import TensorBoard

In [53]:
tfb = TensorBoard('object_detection')

In [54]:
history = model.fit(x=x_train, y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])    

Epoch 1/100
35/35 [==============================] - 60s 1s/step - loss: 0.1414 - val_loss: 0.1281
Epoch 2/100
35/35 [==============================] - 50s 1s/step - loss: 0.1151 - val_loss: 0.0962
Epoch 3/100
35/35 [==============================] - 49s 1s/step - loss: 0.1000 - val_loss: 0.1221
Epoch 4/100
35/35 [==============================] - 50s 1s/step - loss: 0.0945 - val_loss: 0.0948
Epoch 5/100
35/35 [==============================] - 48s 1s/step - loss: 0.0824 - val_loss: 0.0915
Epoch 6/100
35/35 [==============================] - 51s 1s/step - loss: 0.0785 - val_loss: 0.0844
Epoch 7/100
35/35 [==============================] - 50s 1s/step - loss: 0.0747 - val_loss: 0.0826
Epoch 8/100
35/35 [==============================] - 49s 1s/step - loss: 0.0749 - val_loss: 0.0789
Epoch 9/100
35/35 [==============================] - 50s 1s/step - loss: 0.0497 - val_loss: 0.0532
Epoch 10/100
35/35 [==============================] - 45s 1s/step - loss: 0.0360 - val_loss: 0.0310
Epoch 11/

In [56]:
history = model.fit(x=x_train, y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)


Epoch 102/200
35/35 [==============================] - 43s 1s/step - loss: 8.7024e-04 - val_loss: 0.0161
Epoch 103/200
35/35 [==============================] - 43s 1s/step - loss: 8.1120e-04 - val_loss: 0.0158
Epoch 104/200
35/35 [==============================] - 44s 1s/step - loss: 6.2074e-04 - val_loss: 0.0159
Epoch 105/200
35/35 [==============================] - 44s 1s/step - loss: 5.1275e-04 - val_loss: 0.0156
Epoch 106/200
35/35 [==============================] - 44s 1s/step - loss: 5.1822e-04 - val_loss: 0.0161
Epoch 107/200
35/35 [==============================] - 45s 1s/step - loss: 4.0451e-04 - val_loss: 0.0159
Epoch 108/200
35/35 [==============================] - 43s 1s/step - loss: 4.3806e-04 - val_loss: 0.0157
Epoch 109/200
35/35 [==============================] - 40s 1s/step - loss: 7.0339e-04 - val_loss: 0.0156
Epoch 110/200
35/35 [==============================] - 43s 1s/step - loss: 5.1075e-04 - val_loss: 0.0154
Epoch 111/200
35/35 [==============================] - 

In [ ]:
# Saving the model
model.save('./models/object_detection.h5')

c:\Users\Asus\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
